# Validation of Simple Models
Simple cobra models with 1-2 bacteria, and 1-2 metabolites

Communities will be:
- I- Single Model, Eat A, Lose Bio 
- II- Single Model, Consume A, Lose B, Eat B, Lose Bio
- III- 2 Model, Eat A, Each Lose Bio
- IV- 2 Model, 1 eat A, 2 eat B 
- V- 2 Model, Eat A, 1 lose Bio&B, 2 eat B, Lose Bio

In [1]:
import iifba
import cobra as cb
import numpy as np
import matplotlib.pyplot as plt

## Create Models

In [2]:
# Sit I, (III-org1, IV-org1)

# Create model
model_I = cb.Model("Org1")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
bio1 = cb.Metabolite("bio1", compartment="c")  # intracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({
    A_e: -1
})
# Biomass exchange: export bio1
exBio1 = cb.Reaction("exchangeBio1", lower_bound=-1000, upper_bound=1000)
exBio1.add_metabolites({
    bio1: -1.0
})

"""#Transporters """
# Transport A_e → A_c
transport = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transport.add_metabolites({
    A_e: -1,
    A_c: 1
})

"""#Internal RXNs"""
# Biomass reaction: A_c → bio1
biomass1 = cb.Reaction("BiomassOrg1", lower_bound=-1000, upper_bound=1000)
biomass1.add_metabolites({
    A_c: -1,
    bio1: 1
})

# Add reactions to the model
model_I.add_reactions([exA, transport, biomass1,  exBio1])

# Set objective
model_I.objective = "exchangeBio1"

# Solve
sol = model_I.optimize()
print(f"Objective direction: {model_I.objective.direction}")
print(f"Max biomass: {sol.objective_value}\n")
print(sol.fluxes)

model_I.summary()

# cb.io.save_json_model(model_I, "sit_I.json")
# cb.io.save_json_model(model_I, "sit_III_1.json")
# cb.io.save_json_model(model_I, "sit_IV_1.json")


Objective direction: max
Max biomass: 10.0

Ex_A           -10.0
T_A             10.0
BiomassOrg1     10.0
exchangeBio1    10.0
Name: fluxes, dtype: float64


Metabolite,Reaction,Flux,C-Number,C-Flux
A_e,Ex_A,10,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
bio1,exchangeBio1,-10,0,0.00%


In [3]:
# Sit II
# Create model
model_II = cb.Model("Org1")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
B_e = cb.Metabolite("B_e", compartment="e")  # extracellular B
B_c = cb.Metabolite("B_c", compartment="c")  # intracellular B
bio1 = cb.Metabolite("bio1", compartment="c")  # intracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A (can import A when available)
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({
    A_e: -1
})
# Exchange reaction: import/export B
exB = cb.Reaction("Ex_B", lower_bound=0, upper_bound=1000)
exB.add_metabolites({
    B_e: -1
})
# Biomass exchange: export bio1
exBio1 = cb.Reaction("exchangeBio1", lower_bound=0, upper_bound=1000)
exBio1.add_metabolites({
    bio1: -1.0
})

"""#Transporters"""
# Transport A_e → A_c
transportA = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transportA.add_metabolites({
    A_e: -1,
    A_c: 1
})
# Transport B_c → B_e
transportB = cb.Reaction("T_B", lower_bound=-1000, upper_bound=1000)
transportB.add_metabolites({
    B_c: -1,
    B_e: 1
})

"""# Internal Rxns"""
# Biomass reaction from A: A_c → bio1 + B_c (always creates some B)
biomassA = cb.Reaction("BiomassFromA", lower_bound=0, upper_bound=1000)
biomassA.add_metabolites({
    A_c: -1,  # Consumes 2 A_c
    bio1: 1,  # Produces bio1
    B_c: 1   # Always produces 1 B_c as byproduct
})
# Biomass reaction from A: A_c → bio1 + B_c (always creates some B)
biomassB = cb.Reaction("BiomassFromB", lower_bound=0, upper_bound=1000)
biomassB.add_metabolites({
    bio1: 1,  # Produces bio1
    B_c: -1   # Always produces 1 B_c as byproduct
})



# Add reactions to the model (FIXED: Added missing biomass1B!)
model_II.add_reactions([exA, exB, transportA, transportB, biomassA, biomassB,  exBio1])



# Set objective
model_II.objective = "exchangeBio1"

# Solve
sol = model_II.optimize()
print(f"Max biomass: {sol.objective_value}")
print(sol.fluxes)

model_II.summary()

# cb.io.save_json_model(model_II, "sit_II.json")


Max biomass: 20.0
Ex_A           -10.0
Ex_B             0.0
T_A             10.0
T_B              0.0
BiomassFromA    10.0
BiomassFromB    10.0
exchangeBio1    20.0
Name: fluxes, dtype: float64


Metabolite,Reaction,Flux,C-Number,C-Flux
A_e,Ex_A,10,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
bio1,exchangeBio1,-20,0,0.00%


In [4]:
# sit III org 2

# Create model
model_III = cb.Model("Org2")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
bio2 = cb.Metabolite("bio2", compartment="c")  # intracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({
    A_e: -1
})
# Biomass exchange: export bio1
exBio2 = cb.Reaction("exchangeBio2", lower_bound=-1000, upper_bound=1000)
exBio2.add_metabolites({
    bio2: -1.0
})

"""#Transporters """
# Transport A_e → A_c
transport = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transport.add_metabolites({
    A_e: -1,
    A_c: 1
})

"""#Internal RXNs"""
# Biomass reaction: A_c → bio1
biomass2 = cb.Reaction("BiomassOrg2", lower_bound=-1000, upper_bound=1000)
biomass2.add_metabolites({
    A_c: -1,
    bio2: 1
})

# Add reactions to the model
model_III.add_reactions([exA, transport, biomass2,  exBio2])

# Set objective
model_III.objective = "exchangeBio2"

# Solve
sol = model_III.optimize()
print(f"Objective direction: {model_III.objective.direction}")

model_III.summary()

# cb.io.save_json_model(model_III, "sit_III_2.json")


Objective direction: max


Metabolite,Reaction,Flux,C-Number,C-Flux
A_e,Ex_A,10,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
bio2,exchangeBio2,-10,0,0.00%


In [5]:
# sit. IV org 2

# Create model
model_IV = cb.Model("Org2")

# Metabolites
B_e = cb.Metabolite("B_e", compartment="e")  # extracellular A
B_c = cb.Metabolite("B_c", compartment="c")  # intracellular A
bio2 = cb.Metabolite("bio2", compartment="c")  # intracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A
exB = cb.Reaction("Ex_B", lower_bound=-10, upper_bound=1000)
exB.add_metabolites({
    B_e: -1
})
# Biomass exchange: export bio1
exBio2 = cb.Reaction("exchangeBio2", lower_bound=-1000, upper_bound=1000)
exBio2.add_metabolites({
    bio2: -1.0
})

"""#Transporters """
# Transport A_e → A_c
transport = cb.Reaction("T_B", lower_bound=-1000, upper_bound=1000)
transport.add_metabolites({
    B_e: -1,
    B_c: 1
})

"""#Internal RXNs"""
# Biomass reaction: A_c → bio1
biomass2 = cb.Reaction("BiomassOrg2", lower_bound=-1000, upper_bound=1000)
biomass2.add_metabolites({
    B_c: -1,
    bio2: 1
})

# Add reactions to the model
model_IV.add_reactions([exB, transport, biomass2,  exBio2])

# Set objective
model_IV.objective = "exchangeBio2"

# Solve
sol = model_IV.optimize()
print(f"Objective direction: {model_IV.objective.direction}")
print(f"Max biomass: {sol.objective_value}\n")
print(sol.fluxes)

model_IV.summary()

# cb.io.save_json_model(model_IV, "sit_IV_2.json")


Objective direction: max
Max biomass: 10.0

Ex_B           -10.0
T_B             10.0
BiomassOrg2     10.0
exchangeBio2    10.0
Name: fluxes, dtype: float64


Metabolite,Reaction,Flux,C-Number,C-Flux
B_e,Ex_B,10,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
bio2,exchangeBio2,-10,0,0.00%


In [6]:
# sit V org 1
# Create model
model_V_1 = cb.Model("Org1")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
B_e = cb.Metabolite("B_e", compartment="e")  # extracellular B
B_c = cb.Metabolite("B_c", compartment="c")  # intracellular B
bio1 = cb.Metabolite("bio1", compartment="c")  # intracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A (can import A when available)
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({
    A_e: -1
})
# Exchange reaction: import/export B
exB = cb.Reaction("Ex_B", lower_bound=0, upper_bound=1000)
exB.add_metabolites({
    B_e: -1
})
# Biomass exchange: export bio1
exBio1 = cb.Reaction("exchangeBio1", lower_bound=0, upper_bound=1000)
exBio1.add_metabolites({
    bio1: -1.0
})

"""#Transporters"""
# Transport A_e → A_c
transportA = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transportA.add_metabolites({
    A_e: -1,
    A_c: 1
})
# Transport B_c → B_e
transportB = cb.Reaction("T_B", lower_bound=-1000, upper_bound=1000)
transportB.add_metabolites({
    B_c: -1,
    B_e: 1
})

"""# Internal Rxns"""
# Biomass reaction from A: A_c → bio1 + B_c (always creates some B)
biomassA = cb.Reaction("BiomassFromA", lower_bound=0, upper_bound=1000)
biomassA.add_metabolites({
    A_c: -1,  # Consumes 2 A_c
    bio1: 1,  # Produces bio1
    B_c: 1   # Always produces 1 B_c as byproduct
})

# Add reactions to the model
model_V_1.add_reactions([exA, exB, transportA, transportB, biomassA,  exBio1])

# Set objective
model_V_1.objective = "exchangeBio1"

# Solve
sol = model_V_1.optimize()
print(f"Max biomass: {sol.objective_value}")
print(sol.fluxes)

model_V_1.summary()

# cb.io.save_json_model(model_V_1, "sit_V_1.json")


Max biomass: 10.0
Ex_A           -10.0
Ex_B            10.0
T_A             10.0
T_B             10.0
BiomassFromA    10.0
exchangeBio1    10.0
Name: fluxes, dtype: float64


Metabolite,Reaction,Flux,C-Number,C-Flux
A_e,Ex_A,10,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
B_e,Ex_B,-10,0,0.00%
bio1,exchangeBio1,-10,0,0.00%


In [7]:
# sit V org 2
# Create model
model_V_2 = cb.Model("Org2")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
B_e = cb.Metabolite("B_e", compartment="e")  # extracellular B
B_c = cb.Metabolite("B_c", compartment="c")  # intracellular B
bio2 = cb.Metabolite("bio2", compartment="c")  # intracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A (can import A when available)
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({
    A_e: -1
})
# Exchange reaction: import/export B
exB = cb.Reaction("Ex_B", lower_bound=-10, upper_bound=1000)
exB.add_metabolites({
    B_e: -1
})
# Biomass exchange: export bio2
exBio2 = cb.Reaction("exchangeBio2", lower_bound=0, upper_bound=1000)
exBio2.add_metabolites({
    bio2: -1.0
})

"""#Transporters"""
# Transport A_e → A_c
transportA = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transportA.add_metabolites({
    A_e: -1,
    A_c: 1
})
# Transport B_c → B_e
transportB = cb.Reaction("T_B", lower_bound=-1000, upper_bound=1000)
transportB.add_metabolites({
    B_c: -1,
    B_e: 1
})

"""# Internal Rxns"""
# Biomass reaction from A: A_c → bio2
biomassA = cb.Reaction("BiomassFromA", lower_bound=0, upper_bound=1000)
biomassA.add_metabolites({
    A_c: -1,  # Consumes A_c
    bio2: 1,  # Produces bio2
})
# Biomass reaction from A: B_c → bio2
biomassB = cb.Reaction("BiomassFromB", lower_bound=0, upper_bound=1000)
biomassB.add_metabolites({
    B_c: -1,  # Consumes 2 B_c
    bio2: 1,  # Produces bio2
})

# Add reactions to the model
model_V_2.add_reactions([exA, exB, transportA, transportB, biomassA, biomassB, exBio2])

# Set objective
model_V_2.objective = "exchangeBio2"

# Solve
sol = model_V_2.optimize()
print(f"Max biomass: {sol.objective_value}")
print(sol.fluxes)

model_V_2.summary()

# cb.io.save_json_model(model_V_2, "sit_V_2.json")


Max biomass: 20.0
Ex_A           -10.0
Ex_B           -10.0
T_A             10.0
T_B            -10.0
BiomassFromA    10.0
BiomassFromB    10.0
exchangeBio2    20.0
Name: fluxes, dtype: float64


Metabolite,Reaction,Flux,C-Number,C-Flux
A_e,Ex_A,10,0,0.00%
B_e,Ex_B,10,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
bio2,exchangeBio2,-20,0,0.00%


In [20]:
# sit VI org 1
# Create model
model_VI_1 = model_V_2.copy()

for rxn in model_VI_1.reactions:
    print(rxn)
    
model_VI_1.metabolites.get_by_id("bio2").id = "bio1"
model_VI_1.reactions.get_by_id("exchangeBio2").id = "exchangeBio1"
model_VI_1.repair()

for rxn in model_VI_1.reactions:
    print(rxn)

cb.io.save_json_model(model_VI_1, "sit_VI_1.json")

Ex_A: A_e <=> 
Ex_B: B_e <=> 
T_A: A_e <=> A_c
T_B: B_c <=> B_e
BiomassFromA: A_c --> bio2
BiomassFromB: B_c --> bio2
exchangeBio2: bio2 --> 
Ex_A: A_e <=> 
Ex_B: B_e <=> 
T_A: A_e <=> A_c
T_B: B_c <=> B_e
BiomassFromA: A_c --> bio1
BiomassFromB: B_c --> bio1
exchangeBio1: bio1 --> 


In [ ]:
# sit VI org 2
# Create model
model_VI_2 = model_V_1.copy()
for rxn in model_VI_2.reactions:
	print(rxn)

model_VI_2.metabolites.get_by_id("bio1").id = "bio2"
model_VI_2.metabolites.get_by_id("A_e").id = "C_e"
model_VI_2.metabolites.get_by_id("A_c").id = "C_c"
model_VI_2.reactions.get_by_id("exchangeBio1").id = "exchangeBio2"
model_VI_2.reactions.get_by_id("T_A").id = "T_C"
model_VI_2.reactions.get_by_id("Ex_A").id = "Ex_C"
model_VI_2.reactions.get_by_id("BiomassFromA").id = "BiomassFromC"
model_VI_2.repair()

for rxn in model_VI_2.reactions:
	print(rxn)

# cb.io.save_json_model(model_VI_2, "sit_VI_2.json")

Ex_A: A_e <=> 
Ex_B: B_e --> 
T_A: A_e <=> A_c
T_B: B_c <=> B_e
BiomassFromA: A_c --> B_c + bio1
exchangeBio1: bio1 --> 
Ex_A: C_e <=> 
Ex_B: B_e --> 
T_C: C_e <=> C_c
T_B: B_c <=> B_e
BiomassFromC: C_c --> B_c + bio2
exchangeBio2: bio2 --> 
